In [1]:
import torch
# Isso vai detectar automaticamente a GPU AMD (via ROCm) ou NVIDIA (via CUDA)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Rodando em: {device}")
block_size = 8
batch_size = 4

Rodando em: cpu


In [6]:
# Comando para ler o conteúdo de um arquivo de texto
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
# Extrai todos os caracteres únicos do texto e os ordena
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [14]:
# Cria mapeamentos de caracteres para inteiros e vice-versa, além de funções de codificação e decodificação
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# Exemplo de lambda
# def add_1(x, y):
#   return x + y

# é o mesmo que 
# add_1 = lambda x, y: x + y
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [15]:
# Basicamente o que estamos fazendo é pegando 80% do total do texto para treinar a LLM (da posição 1 até a posição n)
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


tensor([177221, 180706,  57824, 113599])
inputs:
tensor([[71, 58, 57,  1, 73, 61, 58,  1],
        [68,  1, 66, 58, 58, 73,  1, 61],
        [55, 71, 68, 68, 64, 72,  9,  1],
        [73, 66, 58, 67, 73, 11,  1, 39]])
targets:
tensor([[58, 57,  1, 73, 61, 58,  1, 29],
        [ 1, 66, 58, 58, 73,  1, 61, 58],
        [71, 68, 68, 64, 72,  9,  1, 55],
        [66, 58, 67, 73, 11,  1, 39, 67]])


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)

    def forward(self, index, targets):
        logits = self.token_embedding_table(index)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

        return logits

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)
